# Post-hoc XAI - Training a Neural Network and explaining it with ANCHOR

In [1]:
import pandas as pd
df = pd.read_csv("wnba_clean.csv")

In [2]:
df

,shot_type,made_shot,shot_value,coordinate_x,coordinate_y,home_score,away_score,qtr,quarter_seconds_remaining,game_seconds_remaining,distance,shot_group,shot_group_encoded,shot_type_encoded
0,Jump Shot,False,0,12,9,0,0,1,571,2371,1.500000e+01,Jump Shot,4,41
1,Turnaround Bank Jump Shot,False,0,-13,0,0,0,1,551,2351,1.300000e+01,Jump Shot,4,24
2,Cutting Layup Shot,True,2,4,2,0,2,1,538,2338,4.472136e+00,Layup,2,11
3,Driving Layup Shot,True,2,-3,0,2,2,1,524,2324,3.000000e+00,Layup,2,15
4,Jump Shot,True,3,-16,21,2,5,1,512,2312,2.640076e+01,Jump Shot,4,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41492,Turnaround Fade Away Jump Shot,False,0,23,5,71,77,4,24,24,2.353720e+01,Jump Shot,4,32
41493,Jump Shot,False,0,-23,3,71,77,4,19,19,2.319483e+01,Jump Shot,4,41
41494,Free Throw - 1 of 2,False,0,-214748365,-214748365,71,77,4,16,16,3.037001e+08,Free Throw,6,45
41495,Free Throw - 2 of 2,True,1,-214748365,-214748365,71,78,4,16,16,3.037001e+08,Free Throw,6,57


In [4]:
# Step 1: Load and Prepare the Dataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from alibi.explainers import AnchorTabular

# Load the WNBA dataset
data_path = 'wnba_clean.csv'
df = pd.read_csv(data_path)

# Define target and features
target = 'made_shot'
features = [
    'coordinate_x', 'coordinate_y', 'distance', 'shot_type_encoded','shot_group_encoded',
    'home_score', 'away_score', 'qtr', 
    'quarter_seconds_remaining', 'game_seconds_remaining'
]

# Split dataset into train and test sets
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Debugging: Check dataset shapes and features
print("X_train shape:", X_train_tensor.shape)
print("X_test shape:", X_test_tensor.shape)
print("Features used:", features)


X_train shape: torch.Size([33197, 10])
X_test shape: torch.Size([8300, 10])
Features used: ['coordinate_x', 'coordinate_y', 'distance', 'shot_type_encoded', 'shot_group_encoded', 'home_score', 'away_score', 'qtr', 'quarter_seconds_remaining', 'game_seconds_remaining']


In [5]:
class AdvancedNN(nn.Module):
    def __init__(self, input_dim):
        super(AdvancedNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.3)
        
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.3)
        
        self.fc3 = nn.Linear(64, 32)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.dropout1(self.relu1(self.bn1(self.fc1(x))))
        x = self.dropout2(self.relu2(self.bn2(self.fc2(x))))
        x = self.relu3(self.fc3(x))
        x = self.sigmoid(self.fc4(x))
        return x


# Initialize model
input_dim = X_train_tensor.shape[1]  # 10 input features
model = AdvancedNN(input_dim)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 20  # Increase epochs for better learning
batch_size = 32

for epoch in range(epochs):
    model.train()
    permutation = torch.randperm(X_train_tensor.size(0))  # Shuffle training data
    for i in range(0, X_train_tensor.size(0), batch_size):
        indices = permutation[i:i + batch_size]
        batch_X, batch_y = X_train_tensor[indices], y_train_tensor[indices]

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_X)

        # Compute loss
        loss = criterion(outputs, batch_y)

        # Backward pass
        loss.backward()

        # Optimize
        optimizer.step()

    # Print loss for each epoch
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}")


Epoch 1/20, Loss: 0.5203
Epoch 2/20, Loss: 0.6282
Epoch 3/20, Loss: 0.8099
Epoch 4/20, Loss: 0.7970
Epoch 5/20, Loss: 0.6113
Epoch 6/20, Loss: 0.5251
Epoch 7/20, Loss: 0.6162
Epoch 8/20, Loss: 0.6218
Epoch 9/20, Loss: 0.6059
Epoch 10/20, Loss: 0.6862
Epoch 11/20, Loss: 0.5104
Epoch 12/20, Loss: 0.6049
Epoch 13/20, Loss: 0.6991
Epoch 14/20, Loss: 0.6149
Epoch 15/20, Loss: 0.5967
Epoch 16/20, Loss: 0.6993
Epoch 17/20, Loss: 0.5057
Epoch 18/20, Loss: 0.5127
Epoch 19/20, Loss: 0.7164
Epoch 20/20, Loss: 0.5742


In [6]:
# Evaluate the model on the test set
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient computation for evaluation
    predictions = model(X_test_tensor)
    predictions = (predictions > 0.5).float()  # Convert probabilities to binary predictions
    accuracy = (predictions == y_test_tensor).float().mean().item()  # Calculate accuracy

print(f"Test Set Accuracy: {accuracy:.4f}")


Test Set Accuracy: 0.6620


# Now lets use some post-hoc techniques for this model


In [28]:
from alibi.explainers import AnchorTabular

# Define a prediction function for the trained neural network
def predict_fn(x):
    preds = model(torch.tensor(x, dtype=torch.float32)).detach().numpy()
    return np.hstack((1 - preds, preds))  # Return probabilities for both classes

# Initialize categorical_names dictionary
categorical_names = {}

# Map encoded categorical features to their categories
categorical_names[3] = list(df['shot_type'].unique())  # Replace 3 with index of 'shot_type_encoded'
categorical_names[4] = list(df['shot_group'].unique())  # Replace 4 with index of 'shot_group_encoded'

# Feature names
feature_names = ['coordinate_x', 'coordinate_y', 'distance', 'shot_type_encoded', 'shot_group_encoded',
                 'home_score', 'away_score', 'qtr', 'quarter_seconds_remaining', 'game_seconds_remaining']

# Initialize the AnchorTabular explainer
explainer = AnchorTabular(predict_fn, feature_names, seed=420)

# Fit the explainer on the training data
explainer.fit(X_train_scaled, categorical_names=categorical_names)

AnchorTabular(meta={
  'name': 'AnchorTabular',
  'type': ['blackbox'],
  'explanations': ['local'],
  'params': {'seed': 420, 'disc_perc': (25, 50, 75)},
  'version': '0.9.6'}
)

## Let's start by explaining the first 5 instances

In [34]:
for i in range(5):  # Explain first 5 instances
    instance = X_test_scaled[i].reshape(1, -1)
    explanation = explainer.explain(instance)
    print(f"Instance {i + 1} Explanation:")
    print(f"Anchor: {explanation.anchor}")
    print(f"Precision: {explanation.precision}")
    print(f"Coverage: {explanation.coverage}\n")

Instance 1 Explanation:
Anchor: ['shot_type_encoded > -1.24', 'distance <= -0.51']
Precision: 0.9917695473251029
Coverage: 0.2348

Instance 2 Explanation:
Anchor: ['shot_type_encoded > -1.24', 'distance <= -0.51']
Precision: 0.99375
Coverage: 0.4856

Instance 3 Explanation:
Anchor: ['shot_type_encoded <= -1.24']
Precision: 0.9690265486725663
Coverage: 0.2631

Instance 4 Explanation:
Anchor: ['shot_type_encoded <= -1.24']
Precision: 0.9695652173913043
Coverage: 0.2701

Instance 5 Explanation:
Anchor: ['shot_type_encoded > -1.24', 'distance <= -0.51']
Precision: 0.9935828877005347
Coverage: 0.2374



## We can see that instance 2 has a 48.6% Coverage which means it explains with 99.4% precision 48.6% of the test set scenarios, generalizing very well for that subgroup
### Let's see what that istance is and what it represents by inverting the scaling of the features:

In [43]:
instance = X_test_scaled[1].reshape(1, -1)
# Reverse scaling of the instance
original_values = scaler.inverse_transform(instance)

# Map the feature names to their original values
real_values = dict(zip(features, original_values[0]))

# Print the real values of the instance
print("\nOriginal Feature Values:")
for feature, value in real_values.items():
    print(f"{feature}: {value:.2f}")



Original Feature Values:
coordinate_x: -7.00
coordinate_y: 25.00
distance: 25.96
shot_type_encoded: 41.00
shot_group_encoded: 4.00
home_score: 9.00
away_score: 13.00
qtr: 1.00
quarter_seconds_remaining: 319.00
game_seconds_remaining: 2119.00


### The results suggest that distance <= 25.96 and shot_type_encoded > 41
### Let's see what shot type is the equivalent of our encoding:

In [87]:
# Add counts for shot_type and shot_type_encoded with a clean index
unique_shot_types = (
    df.groupby(['shot_type', 'shot_type_encoded'])
    .size()
    .reset_index(name='count')
    .sort_values('shot_type_encoded')
    .set_index(['shot_type', 'shot_type_encoded'])
)
print("Unique Shot Types and Encoded Values with Counts:")
print(unique_shot_types)

Unique Shot Types and Encoded Values with Counts:
                                                      count
shot_type                          shot_type_encoded       
Tip Shot                           0                    149
Driving Dunk Shot                  1                      1
Running Dunk Shot                  2                      1
Layup Shot Putback                 3                    988
Layup Running Reverse              4                     43
Cutting Finger Roll Layup Shot     5                     47
Layup Driving Reverse              6                    219
Reverse Layup Shot                 7                    241
Running Finger Roll Layup          8                     83
Alley Oop Layup Shot               9                     59
Running Layup Shot                 10                  1561
Cutting Layup Shot                 11                  1641
Running Alley Oop Layup Shot       12                    13
Driving Finger Roll Layup          13             

### Almost every shot (besides 1) of the shot_types_encoded > 41 are free throws.
#### So it's safe to conclude that for every free throw with distance <= 25.96 feet the model predicts correctly 99.4% of the time, given this test set

## Let's try to select an instance close to the decision boundary, it should give a very low coverage:

In [42]:
with torch.no_grad():
    predictions = model(X_test_tensor).squeeze().numpy()  # Get probabilities
    selected_instance_idx = np.argmin(np.abs(predictions - 0.5))  # Closest to 0.5

instance = X_test_scaled[selected_instance_idx].reshape(1, -1)

explanation = explainer.explain(instance)

# Print the explanation
print("\nAnchor Explanation:")
print(f"Anchor: {explanation.anchor}")
print(f"Precision: {explanation.precision}")
print(f"Coverage: {explanation.coverage}")


Anchor Explanation:
Anchor: ['game_seconds_remaining <= 0.01', 'shot_type_encoded <= 0.52', 'away_score <= -0.03', 'home_score <= -0.03']
Precision: 0.9634888438133874
Coverage: 0.0083


### Coverage below 1%, as expected

## Let's try now a shot with a very small distance to the hoop

In [74]:
# Find the index of the instance with the smallest distance
low_distance_idx = np.argmin(X_test_original[:, 2])  # Assuming 'distance' is the 3rd feature

# Select the instance
low_distance_instance = X_test_scaled[low_distance_idx].reshape(1, -1)

low_distance_original = X_test_original[low_distance_idx]


In [80]:
# Filter instances with distance below a threshold (e.g., 3 feet)
threshold = 3  # Example threshold for very low distance
low_distance_mask = X_test_original[:, features.index('distance')] < threshold

# Ensure at least one instance satisfies the condition
if np.any(low_distance_mask):
    # Select the first instance matching the condition
    low_distance_instance = X_test_scaled[low_distance_mask][0].reshape(1, -1)
    low_distance_original = X_test_original[low_distance_mask][0]
else:
    print(f"No instance found with distance below {threshold} feet.")
    low_distance_instance = None

In [83]:
if low_distance_instance is not None:
    # Reverse scaling of the instance
    original_values = scaler.inverse_transform(low_distance_instance)

    # Map the feature names to their original values
    real_values = dict(zip(features, original_values[0]))

    # Print the real values of the instance
    print("\nOriginal Feature Values (Low-Distance Instance):")
    for feature, value in real_values.items():
        print(f"{feature}: {value:.2f}")

    # Generate an explanation for the low-distance instance
    low_distance_explanation = explainer.explain(low_distance_instance)

    # Print the explanation
    print("\nAnchor Explanation for Low-Distance Instance:")
    print(f"Anchor: {low_distance_explanation.anchor}")
    print(f"Precision: {low_distance_explanation.precision}")
    print(f"Coverage: {low_distance_explanation.coverage}")



Original Feature Values (Low-Distance Instance):
coordinate_x: -0.00
coordinate_y: 1.00
distance: 1.00
shot_type_encoded: 11.00
shot_group_encoded: 2.00
home_score: 23.00
away_score: 21.00
qtr: 2.00
quarter_seconds_remaining: 471.00
game_seconds_remaining: 1671.00

Anchor Explanation for Low-Distance Instance:
Anchor: ['shot_type_encoded <= -1.24']
Precision: 0.9779735682819384
Coverage: 0.2719


### The given instance was executed on a 1 feet distance and it represents a layup shot.
### All shots of the test set that are layups, hook shots, Jump Shot, Dunk, tip shot, or other, are classified with 97.7% accuracy in this test set